# 문제 6

(kaggle 형 문제) prob4 를 바탕으로 아래 데이터셋을 만든다.

. prob6_train: DateHour 변수 기준으로 2021년 8월 14일 전(8월 14일 미포함) 데이터 (행의 수: 5256개)

. prob6_test: DateHour 변수 기준으로 2021년 8월 14일 이후(8월 14일 포함) 데이터 (행의 수: 744 개)

일 때, prob6_train으로 target을 예측하는 모델을 만들어, 

prob6_test의 target에 대한 MAE를 최소화하는 모델을 만든다. 

prob6_test의 예측 결과를 아래와 같은 형식으로 출력한다. 파일명은 answer6.csv 이다.

|DateHour|TotalHour|
|--------|---------|
|2021-08-14 00:00:00|102.607580|
|2021-08-14 01:00:00|94.078890|
....

# 시험장 환경 정보

Python: 3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]

|모듈|버젼|
|----|----|
|pandas|0.25.1|
|numpy|1.18.5|
|sklearn|0.21.3|
|scipy|1.5.2|
|mlxtend|0.15.0.0|
|statsmodels|0.11.1|

**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1


In [2]:
df_elec = pd.read_csv('elec.csv', parse_dates=['Date', 'DateHour'])
df_info = pd.read_csv('info.csv')
df_info['Date'] = pd.to_datetime(df_info['Date'])

In [3]:
df_elec1 = df_elec.pivot(
    index='DateHour', 
    columns='Minute', 
    values='Value').reset_index() # index에 위치한 DateHour를 컬럼에 위치시킵니다
df_elec1

Minute,DateHour,15분,30분,45분,60분
0,2021-01-01 00:00:00,62,61,61,61
1,2021-01-01 01:00:00,96,93,116,113
2,2021-01-01 02:00:00,106,96,106,107
3,2021-01-01 03:00:00,92,110,110,109
4,2021-01-01 04:00:00,108,105,106,108
...,...,...,...,...,...
6163,2021-09-14 19:00:00,152,151,171,139
6164,2021-09-14 20:00:00,124,130,128,130
6165,2021-09-14 21:00:00,134,130,125,124
6166,2021-09-14 22:00:00,100,109,120,114


In [4]:
holi = pd.to_datetime(["2021-01-01", "2021-02-11", "2021-02-12", "2021-03-01", "2021-05-05", "2021-05-19", "2021-08-16"]).date

# pd.Series.dt accessor를 통해 파생 변수들을 만듭니다.
df_elec1 = df_elec1.assign(
    DayName = df_elec1['DateHour'].dt.weekday, 
    Hour = df_elec1['DateHour'].dt.hour,
    AM = (df_elec1['DateHour'].dt.hour >= 12).astype('int'),
    Weekend_yn = df_elec1['DateHour'].dt.weekday.isin([5, 6]).astype('int'),
    Holiday_yn = df_elec1['DateHour'].dt.date.isin(holi).astype('int'), # df_elec1['DateHour'].dt.date는 pd.Series 입니다.
    Avg = df_elec1.mean(axis=1),
    TotalHour = df_elec1.sum(axis=1),
)

In [5]:
df_info1 = df_info.fillna(0)

In [6]:
df_basetable1 = df_elec1.merge(
    df_info1, 
    left_on='DateHour', 
    right_on='Date', how='inner'
).drop(columns='Date') # DateHour가 컬럼에 있으므로 DateHour, Date 모두 존재하여, Date는 삭제했습니다.
display(df_basetable1.head())
df_basetable1.shape

,DateHour,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,Avg,TotalHour,생산량,기온,풍속,습도,강수량,전기요금(계절),공장인원,인건비
0,2021-01-01 00:00:00,62,61,61,61,4,0,0,0,1,61.25,245,0,-3.2,2.4,71,0.0,109.8,0.0,1.5
1,2021-01-01 01:00:00,96,93,116,113,4,1,0,0,1,104.50,418,0,-4.5,1.5,77,0.0,109.8,0.0,1.5
2,2021-01-01 02:00:00,106,96,106,107,4,2,0,0,1,103.75,415,0,-3.9,2.6,58,0.0,109.8,0.0,1.5
3,2021-01-01 03:00:00,92,110,110,109,4,3,0,0,1,105.25,421,0,-4.1,2.6,56,0.0,109.8,0.0,1.5
4,2021-01-01 04:00:00,108,105,106,108,4,4,0,0,1,106.75,427,0,-4.6,2.6,60,0.0,109.8,0.0,1.5


(6168, 20)

In [7]:
# shift를 이용합니다.
df_prob3 = df_basetable1.sort_values('DateHour')
df_prob3['target'] = df_prob3['TotalHour'].shift(-24)
df_prob4 = pd.concat(
    [df_prob3] + [
        df_prob3['TotalHour'].shift(24 * i).rename('lag_{}'.format(i)) for i in range(1, 7)
    ], axis=1
).dropna()
df_prob4.head()

,DateHour,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,...,전기요금(계절),공장인원,인건비,target,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6
144,2021-01-07 00:00:00,22,22,22,25,3,0,0,0,0,...,109.8,0.000000,1.5,252.0,96.0,96.0,271.0,88.0,253.0,245.0
145,2021-01-07 01:00:00,26,22,22,22,3,1,0,0,0,...,109.8,0.369565,1.5,396.0,85.0,85.0,432.0,99.0,418.0,418.0
146,2021-01-07 02:00:00,22,21,21,22,3,2,0,0,0,...,109.8,0.000000,1.5,411.0,85.0,85.0,439.0,88.0,415.0,415.0
147,2021-01-07 03:00:00,22,22,22,23,3,3,0,0,0,...,109.8,0.629213,1.5,398.0,90.0,90.0,426.0,88.0,421.0,421.0
148,2021-01-07 04:00:00,22,26,26,23,3,4,0,0,0,...,109.8,2.865979,1.5,420.0,93.0,93.0,435.0,95.0,427.0,427.0


In [8]:
# shift를 사용하지 않고 target를 구하는 방법입니다. (값을 구하기만 하고, 따로 prob4로 만들지는 않았습니다.)
df_basetable1.join(
    df_basetable1[['DateHour','TotalHour']].assign(
        DateHour=lambda x: x['DateHour'] - pd.Timedelta(days=1)
    ).set_index('DateHour').rename(columns={'TotalHour': 'target'}),
    on='DateHour', how='inner'
)

,DateHour,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,...,TotalHour,생산량,기온,풍속,습도,강수량,전기요금(계절),공장인원,인건비,target
0,2021-01-01 00:00:00,62,61,61,61,4,0,0,0,1,...,245,0,-3.2,2.4,71,0.0,109.8,0.000000,1.5,253
1,2021-01-01 01:00:00,96,93,116,113,4,1,0,0,1,...,418,0,-4.5,1.5,77,0.0,109.8,0.000000,1.5,418
2,2021-01-01 02:00:00,106,96,106,107,4,2,0,0,1,...,415,0,-3.9,2.6,58,0.0,109.8,0.000000,1.5,415
3,2021-01-01 03:00:00,92,110,110,109,4,3,0,0,1,...,421,0,-4.1,2.6,56,0.0,109.8,0.000000,1.5,421
4,2021-01-01 04:00:00,108,105,106,108,4,4,0,0,1,...,427,0,-4.6,2.6,60,0.0,109.8,0.000000,1.5,427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6139,2021-09-13 19:00:00,162,160,148,122,0,19,1,0,0,...,592,2820,22.3,2.1,78,0.0,167.2,4.763514,1.5,613
6140,2021-09-13 20:00:00,113,122,122,126,0,20,1,0,0,...,483,14,22.2,1.3,78,0.0,167.2,0.028986,1.5,512
6141,2021-09-13 21:00:00,122,131,122,115,0,21,1,0,0,...,490,72,21.9,1.9,77,0.0,167.2,0.146939,1.5,513
6142,2021-09-13 22:00:00,97,108,122,113,0,22,1,0,0,...,440,11,21.7,1.4,77,0.0,167.2,0.025000,1.5,443


In [10]:
# 문제에서 제시한 데이터셋을 만듭니다.
df_prob6_train = df_prob4.loc[df_prob4['DateHour'] < '2021-08-14']
df_prob6_test = df_prob4.loc[df_prob4['DateHour'] >= '2021-08-14']
# 정답 target은 떼어냅니다.
s_kaggle_ans =  df_prob6_test.pop('target')